In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch
import pandas as pd
from datasets import load_dataset

dataset = load_dataset("esnli")

tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = AutoModelWithLMHead.from_pretrained("t5-base")


In [3]:
max_source_length = len(max(dataset['train']['premise'], key=len)) + len(max(dataset['train']['hypothesis'], key=len))
max_target_length = 512

In [7]:
train = dataset['train']
input_seqs = []
output_seqs = []
cnt = 0
for row in train:
    if cnt > 100000:
        break
    input_seqs.append(f"explain: premise: {row['premise']} hypothesis: {row['hypothesis']}</s>")
    output_seqs.append(row['explanation_1'])
    cnt += 1

In [8]:
encoding = tokenizer(input_seqs, padding='longest', max_length=max_source_length, truncation=True, return_tensors='pt')
input_ids, attention_mask = encoding.input_ids, encoding.attention_mask

target_encoding = tokenizer(output_seqs, padding='longest', max_length=max_target_length, truncation=True, return_tensors='pt')
labels = target_encoding.input_ids
labels = torch.tensor(labels)
labels[labels == tokenizer.pad_token_id] = -100

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [ ]:
test = dataset['test'][0]
input_seq = f"explain: premise: {test['premise']} hypothesis: {test['hypothesis']}</s>"
test_input = tokenizer(input_seq, return_tensors='pt').input_ids
outputs = model.generate(test_input)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))